In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import dask
import numpy as np
import pandas as pd
import xarray as xr
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
from os.path import join as pjoin
import plotly.graph_objects as go
from scipy.stats import zscore, skew

sys.path.append('../')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf

In [ ]:
## Settings
project_folder = ['MultiCon_Imaging']
experiment_folders = ['MultiCon_Imaging5', 'MultiCon_Imaging6', 'MultiCon_Imaging7']
dpath = f'../../{project_folder[0]}'
fig_path = f'../../../Manuscripts/MultiCon/intermediate_plots/reward_activity'
chance_color = '#7d7d7d'
avg_color = '#287347'
subject_color = '#7d7d7d'
ce_colors = ['#7A22BC', '#378616']
ce_colors_dict = {'Two-context': '#378616', 'Multi-context': '#7A22BC'}
symbol_dict = {'Two-context': 'x', 'Multi-context': 'circle'}
symbols_list = ['x', 'circle']
context_colors = {'A': '#00802d', 'B': '#006c79', 'C': '#004da4', 'D': '#430073'}
mouse_colors = ['midnightblue', 'darkred', 'darkorchid', 'darkturquoise']
sem_color = 'rgba(40, 115, 71, 0.6)'
male_mice = ['mc44', 'mc46', 'mc54', 'mc55', 'mc64', 'mc65']
control_mice = ['mc46', 'mc49', 'mc52', 'mc54', 'mc59', 'mc60', 'mc61', 'mc64']
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]
control_list = [f'A{x}' for x in np.arange(1, 16)] + [f'B{x}' for x in np.arange(1, 6)]
day_list = [f'Day {x}' for x in np.arange(1, 21)]
velocity_thresh = 10
centroid_distance = 4
window_size = 90 ## in frames
sampling_rate = 1/30
correct_size = (window_size * 2) + 1 ## since inclusive on both ends
xaxis = np.arange(-(window_size * sampling_rate), (window_size * sampling_rate)+sampling_rate, sampling_rate)

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

xr.set_options(keep_attrs=True)

## Create cluster
cluster = LocalCluster(n_workers=25,
                       memory_limit='3GB',
                       resources={'MEM': 1},
                       threads_per_worker=2,
                       dashboard_address=':8787')
client = Client(cluster)

## Set seed
np.random.seed(24601)

### Testing for one mouse.

In [ ]:
mouse = 'mc46'
session = '1'
C_path = f'../../{project_folder[0]}/{experiment_folders[0]}/output/aligned_minian/{mouse}/C/{mouse}_C_{session}.nc'

C = xr.open_dataset(C_path)['C']

## Normalize data
zdata = xr.apply_ufunc(
        zscore,
        C.chunk({'frame': -1, 'unit_id': 50}),
        input_core_dims=[['frame']],
        output_core_dims=[['frame']],
        kwargs={'axis': 1},
        dask='parallelized'
).compute()

In [ ]:
## Activity around both rewards averaged together
windowed_data = np.empty((zdata.shape[0], correct_size))
windowed_data.fill(np.nan)
windowed_sem = np.empty((zdata.shape[0], correct_size))
windowed_sem.fill(np.nan)

rewards = zdata[:, zdata['water']]
for unit in np.arange(0, zdata.shape[0]):
    nan_array = np.empty((rewards.shape[1], correct_size))
    nan_array.fill(np.nan)
    for idx in np.arange(0, rewards.shape[1]):
        udata = zdata[unit, :]
        d = udata[(udata['frame'] >= rewards[:, idx]['frame'] - window_size) & (udata['frame'] <= rewards[:, idx]['frame'] + window_size)]
        nan_array[idx] = d.values
        avg_response = np.mean(nan_array, axis=0)
        sem = np.std(nan_array, axis=0, ddof=1) / rewards.shape[1]
    windowed_data[unit] = avg_response
    windowed_sem[unit] = sem

In [ ]:
## Plot zscored activity around reward delivery for a single neuron
neuron = 3
fig = pf.custom_graph_template(x_title='Time from Reward (s)', y_title='Average Z-Scored Activity', titles=[f'Neuron {neuron}'])
fig.add_trace(go.Scatter(x=xaxis, y=windowed_data[neuron], mode='lines', line_color=avg_color, showlegend=False))
fig.add_trace(go.Scatter(x=xaxis, y=windowed_data[neuron] + windowed_sem[neuron], mode='lines', 
                         name='Upper Bound', line=dict(width=0), showlegend=False, line_color=sem_color))
fig.add_trace(go.Scatter(x=xaxis, y=windowed_data[neuron] - windowed_sem[neuron], mode='lines', line_color=sem_color,
                         name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=sem_color, fill='tonexty'))
fig.add_vline(x=0, line_dash='dash', line_width=1, line_color='darkgrey', opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, f'example_reward_activity_neuron{neuron}.png'), width=500, height=500)

In [ ]:
## Look at response of all neurons around reward, sorted by peak activity
centers = np.argmax(windowed_data, axis=1)
order = np.argsort(centers)
fig = pf.custom_graph_template(x_title='Time from Reward (s)', y_title='Neuron')
fig.add_trace(go.Heatmap(x=xaxis, y=np.arange(1, windowed_data.shape[0]+1), z=windowed_data[order], colorscale='magma'))
fig.add_vline(x=0, line_dash='dash', line_width=1, line_color='darkgrey', opacity=1)
fig.update_yaxes(autorange='reversed')
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_{session}_all_cells_time_from_reward_3s.png'), width=500, height=500)

In [ ]:
## Look at response of all neurons around reward, sorted by minimum activity
centers = np.argmin(windowed_data, axis=1)
order = np.argsort(centers)
fig = pf.custom_graph_template(x_title='Time from Reward (s)', y_title='Neuron')
fig.add_trace(go.Heatmap(x=xaxis, y=np.arange(1, windowed_data.shape[0]+1), z=windowed_data[order], colorscale='magma'))
fig.add_vline(x=0, line_dash='dash', line_width=1, line_color='darkgrey', opacity=1)
fig.update_yaxes(autorange='reversed')
fig.show()

In [ ]:
## Average population activity
fig = pf.custom_graph_template(x_title='Time from Reward (s)', y_title='Average Z-Scored Activity')
fig.add_trace(go.Scatter(x=xaxis, y=np.mean(windowed_data, axis=1), mode='lines', line_color='midnightblue'))
fig.add_vline(x=0, line_dash='dash', line_width=1, line_color='darkgrey', opacity=1)
fig.show()

### For the mouse above, separate activity around the two rewards.

In [ ]:
## Activity around both reward ports
windowed_data, windowed_sem, rw_pre, rw_post, rw_one_activity, rw_two_activity = ctn.reward_activity(zdata, correct_size=correct_size, window_size=window_size,
                                                                                                     reward_one=zdata.attrs['reward_one'], reward_two=zdata.attrs['reward_two'])

In [ ]:
## Shuffled distribution of reward activity
df = ctn.compute_shuffled_rw_diff(zdata, correct_size=correct_size, window_size=window_size, num_simulations=500)
df

In [ ]:
## Measure reward pos/neg modulated neurons
percentiles = (5, 95)
num_neurons = C.shape[0]
rw_one_diff = rw_post[:, 0] - rw_pre[:, 0]
rw_two_diff = rw_post[:, 1] - rw_pre[:, 1]
rw_one_pos = np.array([rw_one_diff[neuron] > np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], percentiles[1]) for neuron in np.arange(0, num_neurons)])
rw_two_pos = np.array([rw_two_diff[neuron] > np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], percentiles[1]) for neuron in np.arange(0, num_neurons)])
rw_one_neg = np.array([rw_one_diff[neuron] < np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], percentiles[0]) for neuron in np.arange(0, num_neurons)])
rw_two_neg = np.array([rw_two_diff[neuron] < np.percentile(df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], percentiles[0]) for neuron in np.arange(0, num_neurons)])

In [ ]:
## Plot zscored activity around reward delivery for a single neuron
## Neuron 3, 18, 29, 52, 73 for mc46 session 5
neuron = 80
fig = pf.custom_graph_template(x_title='Time from Reward (s)', y_title='Averaged Z-Scored Activity', titles=[f'Neuron {neuron} R1', f'Neuron {neuron} R2'],
                               rows=1, columns=2, shared_x=True, shared_y=True, width=600, font_size=26, master_axes=True)
for ridx in [0, 1]:
    fig.add_trace(go.Scatter(x=xaxis, y=windowed_data[neuron, :, ridx], mode='lines', line_color=avg_color, showlegend=False), row=1, col=ridx + 1)
    fig.add_trace(go.Scatter(x=xaxis, y=windowed_data[neuron, :, ridx] + windowed_sem[neuron, :, ridx], mode='lines', 
                            name='Upper Bound', line=dict(width=0), showlegend=False, line_color=sem_color), row=1, col=ridx + 1)
    fig.add_trace(go.Scatter(x=xaxis, y=windowed_data[neuron, :, ridx] - windowed_sem[neuron, :, ridx], mode='lines', line_color=sem_color,
                            name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=sem_color, fill='tonexty'), row=1, col=ridx + 1)
fig.add_vline(x=0, line_dash='dash', line_width=1, line_color='darkgrey', opacity=1)
fig.update_yaxes(range=[-0.5, 3.3])
if session == '1':
    fig.update_yaxes(range=[-0.5, 10])
fig.show()
# fig.write_image(pjoin(fig_path, f'{mouse}_{session}_neuron{neuron}_r1r2act.png'), width=600, height=500)

In [ ]:
## Plot heatmaps of activity during reward delivery at both ports for a given unit specified in the code block above
fig = pf.custom_graph_template(x_title='Time from Reward (s)', y_title='Reward', rows=1, columns=2, shared_x=True, shared_y=True,
                               titles=[f'Neuron {neuron} R1', f'Neuron {neuron} R2'], master_axes=True, width=800)
for rw in [0, 1]:
    if rw == 0:
        pdata = rw_one_activity
    else:
        pdata = rw_two_activity
    fig.add_trace(go.Heatmap(x=xaxis, y=np.arange(1, pdata.shape[1]+1), z=pdata[:, :, neuron], coloraxis='coloraxis1'), row=1, col=rw + 1)
fig.add_vline(x=0, line_dash='dash', line_width=1, line_color='darkgrey', opacity=1)
fig.update_yaxes(autorange='reversed')
fig.update_layout(coloraxis = {'colorscale':'magma'})
fig.show()
# fig.write_image(pjoin(fig_path, f'{mouse}_{session}_neuron{neuron}_rw1rw2heatmaps.png'), width=800, height=500)

In [ ]:
## Cell activity across time
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)')
fig.add_trace(go.Scattergl(x=C['behav_t'], y=C.values[neuron, :], mode='lines', line_color='darkgrey'))
fig.show()

In [ ]:
## Plot shuffle distributions
fig = pf.custom_graph_template(x_title='Activity Difference', y_title='', titles=[f'Neuron {neuron} R1', f'Neuron {neuron} R2'],
                               rows=1, columns=2, shared_x=True, shared_y=True, width=800)
fig.add_trace(go.Histogram(x=df['difference'][(df['unit'] == neuron) & (df['dim'] == 0)], marker_color='darkgrey', marker_line_width=1, 
                           marker_line_color='black', histnorm='probability', showlegend=False), row=1, col=1)
fig.add_trace(go.Histogram(x=df['difference'][(df['unit'] == neuron) & (df['dim'] == 1)], marker_color='darkgrey', marker_line_width=1, 
                           marker_line_color='black', histnorm='probability', showlegend=False), row=1, col=2)
fig.add_vline(x=rw_post[neuron, 0] - rw_pre[neuron, 0], line_color='red', line_dash='dash', line_width=1, opacity=1, row=1, col=1)
fig.add_vline(x=rw_post[neuron, 1] - rw_pre[neuron, 1], line_color='red', line_dash='dash', line_width=1, opacity=1, row=1, col=2)
fig.update_yaxes(title='Probability', col=1)
fig.show()
# fig.write_image(pjoin(fig_path, f'{neuron}_rw_shuffle_distributions.png'), width=800, height=500)

In [ ]:
client.close()
cluster.close()